In [1]:
import pandas as pd
import difflib
from tqdm import tqdm
import json
import os
import subprocess
import tempfile
import ast

In [2]:
# Load the dataset from the Excel file
file_path = "/home/odedh/SML-For-Debug/data/dataset/PyresBugs.xlsx"  # Replace with the actual file path
df = pd.read_excel(file_path)

df = df[["Fixed_Method", "Faulty Code", "Fault Free Code"]].dropna().reset_index(drop=True)
df.head(10)

,Fixed_Method,Faulty Code,Fault Free Code
0,"_make_sqlite_account_info(self, env=None, last...","def _make_sqlite_account_info(self, env=None, ...","def _make_sqlite_account_info(self, env=None, ..."
1,"def _validate_source(source, req)","def _validate_source(source, req):\n\t\tif sou...","def _validate_source(source, req):\n\t\tif sou..."
2,"def _single_run(self, application, sock)","def _single_run(self, application, sock):\n\t\...","def _single_run(self, application, sock):\n\t\..."
3,"def get_api_response_ext(self, http_resp, url=...","def get_api_response_ext(self, http_resp, url=...","def get_api_response_ext(self, http_resp, url=..."
4,def delete(self),"def delete(self):\n\t\tself._run_command(""dele...","def delete(self):\n\t\tself._run_command([""del..."
5,def run_server(self),def run_server(self):\n\t\tif cfg.CONF.pydev_w...,def run_server(self):\n\t\tif cfg.CONF.pydev_w...
6,"def parse_uri(self, uri)","def parse_uri(self, uri):\n\t\tif not uri.star...","def parse_uri(self, uri):\n\t\tvalid_schema = ..."
7,"def execute(self, file_path, **kwargs)","def execute(self, file_path, **kwargs):\n\t\tt...","def execute(self, file_path, **kwargs):\n\t\tt..."
8,"def set_acls(self, location, public=False, rea...","def set_acls(self, location, public=False, rea...","def set_acls(self, location, public=False, rea..."
9,"def _run_command(self, command, data, *params)","def _run_command(self, command, data, *params)...","def _run_command(self, command, data, *params)..."


In [5]:
save_interval = 50  # You can change this value as needed
diff_df_records = []

def unescape_python_code(code_str: str) -> str:
    """Safely convert escaped newlines into actual code lines."""
    try:
        return ast.literal_eval(f"'''{code_str}'''")
    except (SyntaxError, ValueError):
        return None  # Return None so we can handle it explicitly later


def git_style_diff(buggy_code: str, fixed_code: str, filename="function.py") -> str:
    with tempfile.TemporaryDirectory() as temp_dir:
        file_a = f"{temp_dir}/a/{filename}"
        file_b = f"{temp_dir}/b/{filename}"

        # Write the buggy and fixed code to two files
        for path, content in [(file_a, buggy_code), (file_b, fixed_code)]:
            os.makedirs(os.path.dirname(path), exist_ok=True)
            with open(path, "w") as f:
                f.write(content)
                
        # Use git diff on the two files
        cmd = ["git", "diff", "--no-index", "--", file_a, file_b]
        result = subprocess.run(cmd, capture_output=True, text=True)
        return result.stdout.strip()


for i, row in tqdm(df.iterrows(), total=len(df)):
    filename = row.get("Function Name", "function.py")
    function_key = row.get("Fixed_Method", f"entry_{i}")

    buggy_code = unescape_python_code(row.get("Faulty Code", ""))
    fixed_code = unescape_python_code(row.get("Fault Free Code", ""))

    # If parsing failed, skip and insert empty diff
    if buggy_code is None or fixed_code is None:
        diff_df_records.append({function_key: ""})
        continue

    try:
        diff_text = git_style_diff(buggy_code, fixed_code, filename)
        # Remove all preceding text before the "@@" symbol
        diff_text = diff_text.split("@@")[2] if "@@" in diff_text else diff_text
        diff_df_records.append({function_key: diff_text})
    except Exception as e:
        print(f"Failed diff at entry {i}: {e}")
        diff_df_records.append({function_key: ""})
        continue

    # Save interim results every N iterations
    if (i + 1) % save_interval == 0:
        with open("/home/odedh/SML-For-Debug/data/dataset2.0/diffs.json", "w") as f:
            json.dump(diff_df_records, f, indent=4)

100%|██████████| 5006/5006 [00:18<00:00, 269.20it/s]
